In [1]:
import numpy as np
import zarr
from datetime import datetime

import sys

if not sys.warnoptions:
    import warnings
    warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
store_path = "/eodc/private/tempearth/s1sig0.zarr"

In [20]:
now = datetime.now()
now_np = np.datetime64(now).astype('datetime64[D]')
now_np = np.datetime64("2025-08-11")
origin = np.datetime64("2014-10-01")
timesteps = int((now_np-origin).astype(int))

In [21]:
x_extent = np.arange(4500000, 5400000, 20)
y_extent = np.arange(1800000, 1200000, -20)
time_extent = np.arange(0,timesteps,1)
rel_orbit_extent = [15, 22, 37, 44, 51, 66, 73, 88, 95, 117, 124, 139, 146, 161, 168, 175]

In [22]:
shape = (16,time_extent.shape[0],y_extent.shape[0],x_extent.shape[0])

chunk_shape = (1,30,500,500)
shard_shape = (1,30,5000,5000)

x_shape = x_extent.shape
y_shape = y_extent.shape
time_shape = time_extent.shape

In [23]:
overwrite=True
store = zarr.storage.LocalStore(store_path)
root = zarr.create_group(store=store, overwrite=overwrite,
                        attributes={"freq": "1D",
                                    "spatial_resolution": 20,
                                    "border_noise_removed": True,
                                    "creator": "gfm_ops",
                                    "crs_wkt": "PROJCS[\"Azimuthal_Equidistant\",GEOGCS[\"WGS 84\",DATUM[\"WGS_1984\",SPHEROID[\"WGS 84\",6378137,298.257223563,AUTHORITY[\"EPSG\",\"7030\"]],AUTHORITY[\"EPSG\",\"6326\"]],PRIMEM[\"Greenwich\",0],UNIT[\"degree\",0.0174532925199433],AUTHORITY[\"EPSG\",\"4326\"]],PROJECTION[\"Azimuthal_Equidistant\"],PARAMETER[\"latitude_of_center\",53],PARAMETER[\"longitude_of_center\",24],PARAMETER[\"false_easting\",5837287.81977],PARAMETER[\"false_northing\",2121415.69617],UNIT[\"metre\",1,AUTHORITY[\"EPSG\",\"9001\"]]]",
                                    "dem_source": "/store_frei/eodc_gfm/datapool/copdem/CopDEM30m_global_mosaic.tif",
                                    "egm_applied": False,
                                    "mode": "IW",
                                    "orbit_file_type": "RESORB",
                                    "slice_gap_filled": False,
                                    "slice_overlap_removed": False,
                                    "snap_version": "8.0",
                                    "spatial_filter": None,
                                    "thermal_noise_removed": True,
                                    "worker_git_commit": None,
                                    "worker_git_tag": "v1.1.2",
                                    "worker_name": "s1-sigma",
                                    "wrapper_git_commit": None,
                                    "wrapper_git_tag": "v1.1.2",
                                    "wrapper_name": "gfm-sigma-nrt",
                                    "AREA_OR_POINT": "Area",
                                    })

In [ ]:
root.create_array(name="VH",
                    shape=shape,
                    shards=shard_shape,
                    chunks=chunk_shape,
                    compressors = zarr.codecs.BloscCodec(),
                    dtype="int16",
                    fill_value=-9999,
                    dimension_names=["relative_orbit_number", "time", "y", "x"],
                    config={"write_empty_chunks":False},
                    attributes={"_FillValue": -9999,
                                "_ARRAY_DIMENSIONS": ["relative_orbit_number", "time", "y", "x"],
                                "scale_factor": 0.1,
                                "description": "Sigma nought radar backscatter coefficient in VH polarisation",
                                "long_name": "Sentinel-1 VH polarization sigma nought backscatter",
                                "standard_name": "radar_backscatter_coefficient",
                                "polarisation": "VH",
                                "units": "dB",
                                "grid_mapping": "spatial_ref"},
                    overwrite=overwrite)

root.create_array(name="VV",
                    shape=shape,
                    shards=shard_shape,
                    chunks=chunk_shape,
                    compressors = zarr.codecs.BloscCodec(),
                    dtype="int16",
                    fill_value=-9999,
                    dimension_names=["relative_orbit_number", "time", "y", "x"],
                    config={"write_empty_chunks":False},
                    attributes={"_FillValue": -9999,
                                "_ARRAY_DIMENSIONS": ["relative_orbit_number", "time", "y", "x"],
                                "scale_factor": 0.1,
                                "description": "Sigma nought radar backscatter coefficient in VV polarisation",
                                "long_name": "Sentinel-1 VV polarization sigma nought backscatter",
                                "standard_name": "radar_backscatter_coefficient",
                                "polarisation": "VV",
                                "units": "dB",
                                "grid_mapping": "spatial_ref"},
                    overwrite=overwrite)

root.create_array(name="sensing_date",
                shape=shape,
                shards=shard_shape,
                chunks=chunk_shape,
                dtype="int64",
                fill_value=-9999,
                dimension_names=["relative_orbit_number","time", "y", "x"],
                attributes={"units": "seconds since 2014-10-01 00:00:00",
                            "calendar": "proleptic_gregorian",
                            "_FillValue": -9999,
                            "_ARRAY_DIMENSIONS": ["relative_orbit_number", "time", "y", "x"],
                            "long_name": "sensing time of measurement",
                            "standard_name": "time",
                            "grid_mapping": "spatial_ref"},
                overwrite=overwrite)

root.create_array(name="absolute_orbit_number",
                shape=shape,
                shards=shard_shape,
                chunks=chunk_shape,
                dtype="int32",
                fill_value=-9999,
                dimension_names=["relative_orbit_number","time", "y", "x"],
                attributes={"_FillValue": -9999,
                            "_ARRAY_DIMENSIONS": ["relative_orbit_number", "time", "y", "x"],
                            "long_name": "Absolute orbit number",
                            "units": "",
                            "grid_mapping": "spatial_ref"},
                overwrite=overwrite)

rel_orbit_number = root.create_array(name="relative_orbit_number",
                shape=(16),
                chunks=(16),
                dtype="int16",
                fill_value=-9999,
                dimension_names=["relative_orbit_number"],
                attributes={"_FillValue": -9999,
                            "long_name": "Relative orbit number",
                            "units": ""},                
                overwrite=overwrite)

spat_ref = group.create_array(name="spatial_ref",
                shape=(1,),
                chunks=(1,),
                shards=(1,),
                compressors = zarr.codecs.BloscCodec(),
                dtype="int8",
                dimension_names=["spatial_ref"],
                attributes={'crs_wkt': 'PROJCS["unknown",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Azimuthal_Equidistant"],PARAMETER["latitude_of_center",53],PARAMETER["longitude_of_center",24],PARAMETER["false_easting",5837287.81977],PARAMETER["false_northing",2121415.69617],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]',
                            'semi_major_axis': 6378137.0,
                            'semi_minor_axis': 6356752.314245179,
                            'inverse_flattening': 298.257223563,
                            'reference_ellipsoid_name': 'WGS 84',
                            'longitude_of_prime_meridian': 0.0,
                            'prime_meridian_name': 'Greenwich',
                            'geographic_crs_name': 'WGS 84',
                            'horizontal_datum_name': 'World Geodetic System 1984',
                            'projected_crs_name': 'unknown',
                            'grid_mapping_name': 'azimuthal_equidistant',
                            'latitude_of_projection_origin': 53.0,
                            'longitude_of_projection_origin': 24.0,
                            'false_easting': 5837287.81977,
                            'false_northing': 2121415.69617,
                            'spatial_ref': 'PROJCS["unknown",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Azimuthal_Equidistant"],PARAMETER["latitude_of_center",53],PARAMETER["longitude_of_center",24],PARAMETER["false_easting",5837287.81977],PARAMETER["false_northing",2121415.69617],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]',
                            'GeoTransform': '4500000.0 20.0 0.0 1800000.0 0.0 -20.0'},
                overwrite=overwrite)

x_array = root.create_array(name="x",
            shape=x_shape,
            chunks=x_shape,
            dtype="float64",
            dimension_names=["x"],
            attributes={"_FillValue": "AAAAAAAA+H8=", #fill value is NaN
                        "axis": "X",
                        "long_name": "x coordinate of projection",
                        "standard_name": "projection_x_coordinate",
                        "units": "m"},
            overwrite=overwrite)

y_array = root.create_array(name="y",
            shape=y_shape,
            chunks=y_shape,
            dtype="float64",
            dimension_names=["y"],
            attributes={"_FillValue": "AAAAAAAA+H8=", #fill value is NaN
                        "axis": "Y",
                        "long_name": "y coordinate of projection",
                        "standard_name": "projection_y_coordinate",
                        "units": "m"},
            overwrite=overwrite)

time_array = root.create_array(name="time",
            shape=time_shape,
            dtype="int64",
            dimension_names=["time"],
            attributes={"units": "days since 2014-10-01",
                        "calendar": "proleptic_gregorian",
                        "long_name": "date of acquisition",
                        "standard_name": "time"},
            overwrite=overwrite)

In [25]:
x_array[:] = x_extent
y_array[:] = y_extent
time_array[:] = time_extent
rel_orbit_number[:] = rel_orbit_extent
zarr.consolidate_metadata(store)

<Group file:///eodc/products/eodc/sentinel1_sig0/s1sig0.zarr>

In [ ]:
now = datetime.now()
now_np = np.datetime64(now).astype('datetime64[D]')
origin = np.datetime64("2014-10-01")

new_shape = int((now_np-origin).astype(int))
new_extent = np.arange(0,new_shape,1)

store = zarr.storage.LocalStore(store_path)
group = zarr.group(store=store)

array_names=set(group.array_keys())
coords = {"time", "x", "y"}
data_arrays = array_names-coords

group["time"].resize(new_shape)
for array in data_arrays:
    group_shape  = group[array].shape
    group[array].resize((new_shape, group_shape[1], group_shape[2]))

zarr.consolidate_metadata(store)

store = zarr.storage.LocalStore(store_path)
group = zarr.group(store=store)

group["time"][:]=new_extent

In [13]:
store = zarr.storage.LocalStore(store_path)
group = zarr.group(store=store)

In [6]:
spat_ref = group.create_array(name="spatial_ref",
                shape=(1,),
                chunks=(1,),
                shards=(1,),
                compressors = zarr.codecs.BloscCodec(),
                dtype="int8",
                dimension_names=["spatial_ref"],
                attributes={'crs_wkt': 'PROJCS["unknown",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Azimuthal_Equidistant"],PARAMETER["latitude_of_center",53],PARAMETER["longitude_of_center",24],PARAMETER["false_easting",5837287.81977],PARAMETER["false_northing",2121415.69617],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]',
                            'semi_major_axis': 6378137.0,
                            'semi_minor_axis': 6356752.314245179,
                            'inverse_flattening': 298.257223563,
                            'reference_ellipsoid_name': 'WGS 84',
                            'longitude_of_prime_meridian': 0.0,
                            'prime_meridian_name': 'Greenwich',
                            'geographic_crs_name': 'WGS 84',
                            'horizontal_datum_name': 'World Geodetic System 1984',
                            'projected_crs_name': 'unknown',
                            'grid_mapping_name': 'azimuthal_equidistant',
                            'latitude_of_projection_origin': 53.0,
                            'longitude_of_projection_origin': 24.0,
                            'false_easting': 5837287.81977,
                            'false_northing': 2121415.69617,
                            'spatial_ref': 'PROJCS["unknown",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Azimuthal_Equidistant"],PARAMETER["latitude_of_center",53],PARAMETER["longitude_of_center",24],PARAMETER["false_easting",5837287.81977],PARAMETER["false_northing",2121415.69617],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]',
                            'GeoTransform': '4500000.0 20.0 0.0 1800000.0 0.0 -20.0'},
                overwrite=True)

In [19]:

zarr.consolidate_metadata(store)

<Group file:///eodc/private/tempearth/s1sig0.zarr>

In [10]:
group["spat_ref"][:] = 0

KeyError: "'spat_ref' not found in consolidated metadata."

In [17]:
group["VH"].attrs["grid_mapping"]="spatial_ref"